# Switching methods of level 2

In [1]:
O = zero_matrix(2)
I= identity_matrix(2)
J = matrix([[1]*2]*2)
Y = 2*I-J

## Infinite family

In [2]:
def R2(m): # matrix R_2m
    def row(i):
        r = []
        for j in range(m):
            if i==j:
                r.append(J)
            elif i==j+1 or (i==0 and j==m-1):
                r.append(Y)
            else:
                r.append(0)
        return r
    return 1/2*block_matrix([row(i) for i in range(m)])

### Patching together adjacency matrices of the switching set

In [3]:
N = matrix([[0,0],[1,1]])
blocks = [O,I,N,N.T] # 2x2 matrices with an even number of ones, up to complementation

In [4]:
good_patches = [] # allowed square submatrices of B matrix of four 2x2 blocks
for B11 in blocks:
    for B12 in blocks:
        for B21 in blocks:
            for B22 in blocks:
                B = block_matrix([[B11,B12],[B21,B22]])
                Bnew = J*B11*J+J*B12*Y+Y*B21*J+Y*B22*Y
                if all(all(i==0 or i==4 for i in r) for r in Bnew.rows()):
                    good_patches.append(B)
len(good_patches)

108

In [5]:
def Bmatrices(m): # adjacency matrices B such that R_2m^T*B*R_2m is and adjacency matrix again
    B = zero_matrix(2*m)
    # backtracking algorithm
    def Bmatrices_rec(B,i,j): # adds a block to B on position (i,j)
        if j==m:
            yield B
        else:
            for block in blocks:
                # fill
                B[[2*i,2*i+1],[2*j,2*j+1]] = block
                B[[2*j,2*j+1],[2*i,2*i+1]] = block.T
                # branching method: check if 4x4 submatrix left below is good
                if B[[2*i,2*i+1,2*i+2,2*i+3],[2*j-2,2*j-1,2*j,2*j+1]] in good_patches:
                    if i==0:
                        for B in Bmatrices_rec(B,j,j+1):
                            yield B
                    else:
                        for B in Bmatrices_rec(B,i-1,j):
                            yield B
    for B in Bmatrices_rec(B,0,1):
        Bnew = R2(m).T*B*R2(m)
        if all(all(i==0 or i==1 for i in r) for r in Bnew.rows()):
            yield matrix(B) # copy of B

In [6]:
list(Bmatrices(2)) # adjacency matrices for GM-switching

[
[0 0 0 0]  [0 0 1 0]
[0 0 0 0]  [0 0 0 1]
[0 0 0 0]  [1 0 0 0]
[0 0 0 0], [0 1 0 0]
]

In [7]:
list(Bmatrices(3)) # adjacency matrices for Six vertex AH-switching

[
[0 0 0 0 0 0]  [0 0 0 0 1 0]  [0 0 1 0 0 1]  [0 0 1 0 1 0]
[0 0 0 0 0 0]  [0 0 0 0 0 1]  [0 0 0 1 0 1]  [0 0 0 1 0 1]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [1 0 0 0 0 0]  [1 0 0 0 1 0]
[0 0 0 0 0 0]  [0 0 0 0 1 1]  [0 1 0 0 0 0]  [0 1 0 0 0 1]
[0 0 0 0 0 0]  [1 0 0 1 0 0]  [0 0 0 0 0 0]  [1 0 1 0 0 0]
[0 0 0 0 0 0], [0 1 0 1 0 0], [1 1 0 0 0 0], [0 1 0 1 0 0],

[0 0 0 0 0 0]  [0 0 0 0 0 1]
[0 0 1 1 0 0]  [0 0 1 1 0 1]
[0 1 0 0 1 0]  [0 1 0 0 0 0]
[0 1 0 0 0 1]  [0 1 0 0 1 1]
[0 0 1 0 0 0]  [0 0 0 1 0 0]
[0 0 0 1 0 0], [1 1 0 1 0 0]
]

### Up to conjugation

In [8]:
def stabiliserR2(m):
    P = zero_matrix(2*m)
    for i in range(m-1):
        P[[2*i,2*i+1],[2*i+2,2*i+3]] = I
    P[[2*m-2,2*m-1],[0,1]] = I
    Z = block_diagonal_matrix([J-I]+[I]*(m-1))
    return [matrix(g) for g in MatrixGroup(P,Z)]

In [9]:
def Brepresentatives(m): # Bmatrices up to conjugation and complementation
    reps = []
    stab = stabiliserR2(m)
    for B in Bmatrices(m):
        Bcomplement = matrix([[1]*2*m]*2*m)-B-identity_matrix(2*m)
        if not any(P.T*B*P in reps or P*Bcomplement*P.T in reps for P in stab):
            reps.append(B)
    return reps

### Checking reducibility

In [10]:
def smaller_switchings(m): # decomposable regular orthogonal 2mx2m matrices of level 2
    for integerPartition in Partitions(m):
        if integerPartition == [m] or list(integerPartition).count(1) > 1:
            continue # since we want only the strictly smaller switchings, and any diagonal block of size 2x2 must be I (by asking that count(1) > 1, we avoid double cases)
        for setPartition in SetPartitions(range(m), integerPartition):
            setPartition = [list(part) for part in setPartition]
            for identityBlock in setPartition+[None]:
                if list(integerPartition).count(1) == 1 and (identityBlock == None or len(identityBlock) != 1):
                    continue
                A = identity_matrix(QQ,2*m)
                for i in range(m):
                    for j in range(m):
                        for part in setPartition:
                            if not part==identityBlock and i in part and j in part:
                                ii = part.index(i)
                                jj = part.index(j)
                                A[[2*i,2*i+1],[2*j,2*j+1]] = R2(len(part))[[2*ii,2*ii+1],[2*jj,2*jj+1]]
                yield A
                # if there are two or more indecomposable blocks of size 6 or larger, we have to take into account the direction of the smaller switching methods
                # we are only going up to size 12, so this only occurs for Twelve vertex switching when there are two blocks of size 6
                if m == 6 and identityBlock == None and list(integerPartition).count(3) == 2: # i.e. two parts of size 3
                    for k in [0,1]:
                        for i in range(m):
                            for j in range(m):
                                if i in setPartition[k] and j in setPartition[k]:
                                    ii = setPartition[k].index(i)
                                    jj = setPartition[k].index(j)
                                    A[[2*i,2*i+1],[2*j,2*j+1]] = (R2(3)^(k+1))[[2*ii,2*ii+1],[2*jj,2*jj+1]] # two times Six vertex AH-switching, in opposite directions
                    yield A

In [11]:
list(smaller_switchings(3)) # Six vertex AH-switching could be reducible to GM-switching

[
[   1    0    0    0    0    0]  [ 1/2  1/2  1/2 -1/2    0    0]
[   0    1    0    0    0    0]  [ 1/2  1/2 -1/2  1/2    0    0]
[   0    0  1/2  1/2  1/2 -1/2]  [ 1/2 -1/2  1/2  1/2    0    0]
[   0    0  1/2  1/2 -1/2  1/2]  [-1/2  1/2  1/2  1/2    0    0]
[   0    0  1/2 -1/2  1/2  1/2]  [   0    0    0    0    1    0]
[   0    0 -1/2  1/2  1/2  1/2], [   0    0    0    0    0    1],

[ 1/2  1/2    0    0  1/2 -1/2]
[ 1/2  1/2    0    0 -1/2  1/2]
[   0    0    1    0    0    0]
[   0    0    0    1    0    0]
[ 1/2 -1/2    0    0  1/2  1/2]
[-1/2  1/2    0    0  1/2  1/2]
]

In [12]:
def reducible(B,m):
    possibleswitches = [identity_matrix(2*m)]
    size = 0
    while size < len(possibleswitches):
        size = len(possibleswitches)
        for R in possibleswitches:
            for S in smaller_switchings(m):
                RS = R*S
                if RS in possibleswitches:
                    continue
                Bnew = RS.T*B*RS
                if all(all(i==0 or i==1 for i in r) for r in Bnew.rows()):
                    if set(RS.columns())==set(R2(m).columns()): # switching (conjugation) gives isomorphic graphs
                        return True
                    possibleswitches.append(RS)
    return False

In [13]:
for m in [2..6]:
    Bs = [B for B in Brepresentatives(m) if not reducible(B,m)]
    print(str(len(Bs)) + " irreducible " + str(2*m) + "x" + str(2*m) + " adjacency matrices:")
    for B in Bs:
        print(B, "\n")

2 irreducible 4x4 adjacency matrices:
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0] 

[0 0 1 0]
[0 0 0 1]
[1 0 0 0]
[0 1 0 0] 

1 irreducible 6x6 adjacency matrices:
[0 0 0 0 0 1]
[0 0 1 1 0 1]
[0 1 0 0 0 0]
[0 1 0 0 1 1]
[0 0 0 1 0 0]
[1 1 0 1 0 0] 

0 irreducible 8x8 adjacency matrices:
3 irreducible 10x10 adjacency matrices:
[0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 1 1 0 1 0 0]
[0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 1 1 0 1]
[0 1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 1 1]
[0 0 0 1 0 0 0 0 0 0]
[1 1 0 1 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0]
[0 0 1 1 0 1 0 0 0 0] 

[0 0 1 0 0 0 0 1 1 0]
[0 0 0 1 1 1 0 1 0 1]
[1 0 0 0 1 0 0 0 0 1]
[0 1 0 0 0 1 1 1 0 1]
[0 1 1 0 0 0 1 0 0 0]
[0 1 0 1 0 0 0 1 1 1]
[0 0 0 1 1 0 0 0 1 0]
[1 1 0 1 0 1 0 0 0 1]
[1 0 0 0 0 1 1 0 0 0]
[0 1 1 1 0 1 0 1 0 0] 

[0 0 0 0 0 1 0 0 0 1]
[0 0 1 1 0 1 1 1 0 1]
[0 1 0 0 0 0 0 1 0 0]
[0 1 0 0 1 1 0 1 1 1]
[0 0 0 1 0 0 0 0 0 1]
[1 1 0 1 0 0 1 1 0 1]
[0 1 0 0 0 1 0 0 0 0]
[0 1 1 1 0 1 0 0 1 1]
[0 0 0 1 0 0 0 1 0 0]
[1 1 0 1 1 1 0 1 0 0] 

18 irreduc

## Fano switching (Seven vertex AH-switching)

In [14]:
Rfano = 1/2*matrix.circulant([-1,1,1,0,1,0,0])
Rfano

[-1/2  1/2  1/2    0  1/2    0    0]
[   0 -1/2  1/2  1/2    0  1/2    0]
[   0    0 -1/2  1/2  1/2    0  1/2]
[ 1/2    0    0 -1/2  1/2  1/2    0]
[   0  1/2    0    0 -1/2  1/2  1/2]
[ 1/2    0  1/2    0    0 -1/2  1/2]
[ 1/2  1/2    0  1/2    0    0 -1/2]

### Adjacency matrices for the switching set

In [15]:
Bmatrices = [] # adjacency matrices B such that Rfano^T*B*Rfano is an adjacency matrix again
B = zero_matrix(7)
# backtracking algorithm
def Bmatrices_rec(B,i,j): # adds a 0 or 1 to B on position (i,j)
    if i==6:
        yield B
    else:
        for entry in [0,1]:
            # fill
            B[i,j] = entry
            B[j,i] = entry
            if j==6:
                for B in Bmatrices_rec(B,i+1,i+2):
                    yield B
            else:
                for B in Bmatrices_rec(B,i,j+1):
                    yield B
for B in Bmatrices_rec(B,0,2): # up to complementation, B[0,1] = 0
    Bnew = Rfano.T*B*Rfano
    if all(all(i==0 or i==1 for i in r) for r in Bnew.rows()):
        Bmatrices.append(matrix(B)) # copy of B
len(Bmatrices)

144

### Up to conjugation

In [16]:
stabFano = []
for p in Permutations(7):
    P = p.to_matrix()
    if set((P.T*Rfano).columns()) == set(Rfano.columns()):
        stabFano.append(P)
len(stabFano)

21

In [17]:
Brepresentatives = [] # Bmatrices up to conjugation and complementation
for B in Bmatrices:
    Bcomplement = matrix([[1]*7]*7)-B-identity_matrix(7)
    if not any(P.T*B*P in Brepresentatives or P.T*Bcomplement*P in Brepresentatives for P in stabFano):
        Brepresentatives.append(B)
len(Brepresentatives)

12

### Checking reducibility

In [18]:
smaller_switchings = [] # either GM- or Six vertex AH-switching
#GM-switching sets are frames (four points, no three on a line)
for s in IntegerModRing(7):
    indices = [s,s+1,s+4,s+6] # frame of PG(2,2), in total there are seven
    A = identity_matrix(QQ,7)
    for i in range(4):
        for j in range(4):
            A[indices[i],indices[j]] = R2(2)[i,j]
    smaller_switchings.append(A)
#AH-switching sets are three lines through a point, without that point
for s in IntegerModRing(7):
    indices = []
    for i in range(7):
        if Rfano[i,s] == 1/2:
            for j in range(7):
                if s != j and Rfano[i,j] == 1/2:
                    indices.append(j)
    A = identity_matrix(QQ,7)
    for i in range(6):
        for j in range(6):
            A[indices[i],indices[j]] = R2(3)[i,j]
    smaller_switchings.append(A)
len(smaller_switchings)

14

In [19]:
def reducible(B):
    possibleswitches = [identity_matrix(7)]
    size = 0
    while size < len(possibleswitches):
        size = len(possibleswitches)
        for R in possibleswitches:
            for S in smaller_switchings:
                RS = R*S
                if RS in possibleswitches:
                    continue
                Bnew = RS.T*B*RS
                if all(all(i==0 or i==1 for i in r) for r in Bnew.rows()):
                    if set(RS.columns())==set(Rfano.columns()): # switching (conjugation) gives isomorphic graphs
                        return True
                    possibleswitches.append(RS)
    return False

In [20]:
Bs = [B for B in Brepresentatives if not reducible(B)]
print(str(len(Bs)) + " irreducible adjacency matrices:")
for B in Bs:
    print(B, "\n")

2 irreducible adjacency matrices:
[0 0 0 0 0 0 1]
[0 0 0 0 1 1 1]
[0 0 0 1 1 1 1]
[0 0 1 0 0 0 1]
[0 1 1 0 0 0 1]
[0 1 1 0 0 0 0]
[1 1 1 1 1 0 0] 

[0 0 0 1 1 0 0]
[0 0 0 0 1 1 0]
[0 0 0 0 0 1 1]
[1 0 0 0 0 0 1]
[1 1 0 0 0 0 0]
[0 1 1 0 0 0 0]
[0 0 1 1 0 0 0] 



The second one is the matrix B1 from "A. Abiad and W.H. Haemers, Cospectral graphs and regular orthogonal matrices of level 2". We check that the first one is equivalent to B3, which has the property that Rfano^T\*B3\*Rfano == Z7\*B3\*Z7 (again, see "A. Abiad and W.H. Haemers, Cospectral graphs and regular orthogonal matrices of level 2").

In [21]:
Z7 = zero_matrix(7)
for i in range(7): Z7[i,6-i] = 1
for B in Bs:
    print(B) # first irreducible matrix
    print()
    for B3 in Bmatrices:
        if Rfano.T*B3*Rfano == Z7*B3*Z7:
            B3complement = matrix([[1]*7]*7)-B3-identity_matrix(7)
            if any(P.T*B3*P==B or P.T*B3complement*P==B for P in stabFano): # B and B3 are equivalent
                print(B3)
                break
    break

[0 0 0 0 0 0 1]
[0 0 0 0 1 1 1]
[0 0 0 1 1 1 1]
[0 0 1 0 0 0 1]
[0 1 1 0 0 0 1]
[0 1 1 0 0 0 0]
[1 1 1 1 1 0 0]

[0 0 0 0 0 1 1]
[0 0 0 0 0 0 1]
[0 0 0 1 0 1 1]
[0 0 1 0 1 0 1]
[0 0 0 1 0 1 0]
[1 0 1 0 1 0 1]
[1 1 1 1 0 1 0]


## Cube switching (sporadic Eight vertex AH-switching)

In [22]:
Rcube = 1/2*block_matrix([[-I,I,I,I],[I,I-J,I,J-I],[I,J-I,I-J,I],[I,I,J-I,I-J]])
Rcube

[-1/2    0  1/2    0  1/2    0  1/2    0]
[   0 -1/2    0  1/2    0  1/2    0  1/2]
[ 1/2    0    0 -1/2  1/2    0    0  1/2]
[   0  1/2 -1/2    0    0  1/2  1/2    0]
[ 1/2    0    0  1/2    0 -1/2  1/2    0]
[   0  1/2  1/2    0 -1/2    0    0  1/2]
[ 1/2    0  1/2    0    0  1/2    0 -1/2]
[   0  1/2    0  1/2  1/2    0 -1/2    0]

In [23]:
neighboursets = [] # can be used to verify the switching conditions
for v in GF(2)^8:
    v = vector(QQ(i) for i in v)
    if all(i==0 or i==1 for i in Rcube.T*v):
        neighboursets.append(v)
        print(v, Rcube.T*v)

(0, 0, 0, 0, 0, 0, 0, 0) (0, 0, 0, 0, 0, 0, 0, 0)
(1, 1, 1, 1, 0, 0, 0, 0) (0, 0, 0, 0, 1, 1, 1, 1)
(1, 1, 0, 0, 1, 1, 0, 0) (0, 0, 1, 1, 0, 0, 1, 1)
(0, 0, 1, 1, 1, 1, 0, 0) (1, 1, 0, 0, 0, 0, 1, 1)
(1, 0, 1, 0, 1, 0, 1, 0) (1, 0, 1, 0, 1, 0, 1, 0)
(0, 1, 0, 1, 1, 0, 1, 0) (1, 0, 0, 1, 0, 1, 1, 0)
(0, 1, 1, 0, 0, 1, 1, 0) (1, 0, 1, 0, 0, 1, 0, 1)
(1, 0, 0, 1, 0, 1, 1, 0) (0, 1, 1, 0, 0, 1, 1, 0)
(0, 1, 1, 0, 1, 0, 0, 1) (1, 0, 0, 1, 1, 0, 0, 1)
(1, 0, 0, 1, 1, 0, 0, 1) (0, 1, 0, 1, 1, 0, 1, 0)
(1, 0, 1, 0, 0, 1, 0, 1) (0, 1, 1, 0, 1, 0, 0, 1)
(0, 1, 0, 1, 0, 1, 0, 1) (0, 1, 0, 1, 0, 1, 0, 1)
(1, 1, 0, 0, 0, 0, 1, 1) (0, 0, 1, 1, 1, 1, 0, 0)
(0, 0, 1, 1, 0, 0, 1, 1) (1, 1, 0, 0, 1, 1, 0, 0)
(0, 0, 0, 0, 1, 1, 1, 1) (1, 1, 1, 1, 0, 0, 0, 0)
(1, 1, 1, 1, 1, 1, 1, 1) (1, 1, 1, 1, 1, 1, 1, 1)


### Adjacency matrices for the switching set

May take several hours...

In [24]:
Bmatrices = [] # adjacency matrices B such that C^T*B*C is and adjacency matrix again
B = zero_matrix(8)
# backtracking algorithm
def Bmatrices_rec(B,i,j): # adds a 0 or 1 to B on position (i,j)
    if i==7:
        yield B
    else:
        for entry in [0,1]:
            # fill
            B[i,j] = entry
            B[j,i] = entry
            if j==7:
                for B in Bmatrices_rec(B,i+1,i+2):
                    yield B
            else:
                for B in Bmatrices_rec(B,i,j+1):
                    yield B
for B in Bmatrices_rec(B,0,2): # up to complementation, B[0,1] = 0
    Bnew = Rcube.T*B*Rcube
    if all(all(i==0 or i==1 for i in r) for r in Bnew.rows()):
        Bmatrices.append(matrix(B)) # copy of B
len(Bmatrices)

752

### Up to conjugation

In [25]:
stabCube = []
for p in Permutations(8):
    P = p.to_matrix()
    if set((P.T*Rcube).columns()) == set(Rcube.columns()):
        stabCube.append(P)
len(stabCube)

24

In [26]:
Brepresentatives = [] # Bmatrices up to conjugation and complementation
for B in Bmatrices:
    Bcomplement = matrix([[1]*8]*8)-B-identity_matrix(8)
    if not any(P.T*B*P in Brepresentatives or P.T*Bcomplement*P in Brepresentatives for P in stabCube):
        Brepresentatives.append(B)
len(Brepresentatives)

40

### Checking reducibility

Smaller switching sets from the infinite family and Fano-switching sets can occur in many ways, so we rather just enumerate all possible permutations of the smaller switching methods, but still up to the symmetries of these smaller switching methods themselves. We do not yet check if they agree with the conditions on the outside vertices.

In [27]:
permutationsAH = []
stabAH = stabiliserR2(3)
for p in Permutations(6):
    P = p.to_matrix()
    if not any(Pstab*P in permutationsAH for Pstab in stabAH):
        permutationsAH.append(P)

In [28]:
permutationsFano = []
for p in Permutations(7):
    P = p.to_matrix()
    if not any(Pstab*P in permutationsFano for Pstab in stabFano):
        permutationsFano.append(P)

In [29]:
smaller_switchings = [] # we first generate all smaller switching sets, also those that are not guaranteed to agree with the conditions on the outside vertices
#GM-switching sets
for indices in Combinations(8,4):
    A = identity_matrix(QQ,8)
    for i in range(4):
        for j in range(4):
            A[indices[i],indices[j]] = R2(2)[i,j]
    smaller_switchings.append(A)
# double GM-switching sets
for setPartition in SetPartitions(range(8), [4,4]):
    indices1 = list(setPartition[0])
    indices2 = list(setPartition[1])
    A = identity_matrix(QQ,8)
    for i in range(4):
        for j in range(4):
            A[indices1[i],indices1[j]] = R2(2)[i,j]
            A[indices2[i],indices2[j]] = R2(2)[i,j]
    smaller_switchings.append(A)
#AH-switching sets
permreps = []
for indices in Combinations(8,6):
    for P in permutationsAH:
        A = identity_matrix(QQ,8)
        for i in range(6):
            for j in range(6):
                A[indices[i],indices[j]] = (P.T*R2(3)*P)[i,j]
        smaller_switchings.append(A)
#Fano switching sets
for indices in Combinations(8,7):
    for P in permutationsFano:
        A = identity_matrix(QQ,8)
        for i in range(7):
            for j in range(7):
                A[indices[i],indices[j]] = (P.T*Rfano*P)[i,j]
        smaller_switchings.append(A)
len(smaller_switchings)

2865

In [30]:
def reducible(B):
    possibleswitches = [identity_matrix(8)]
    size = 0
    while size < len(possibleswitches):
        size = len(possibleswitches)
        for R in possibleswitches:
            for S in smaller_switchings:
                RS = R*S
                if RS in possibleswitches:
                    continue
                Bnew = RS.T*B*RS
                if all(all(i==0 or i==1 for i in r) for r in Bnew.rows()) and all(all(i==0 or i==1 for i in RS.T*v) for v in neighboursets): # check that it agrees with the conditions on the outside vertices
                    if set(RS.columns())==set(Rcube.columns()): # switching (conjugation) gives isomorphic graphs
                        return True
                    possibleswitches.append(RS)
    return False

In [31]:
Bs = [B for B in Brepresentatives if not reducible(B)]
print(str(len(Bs)) + " irreducible adjacency matrices:")
for B in Bs:
    print(B, "\n")

0 irreducible adjacency matrices:


So they are all reducible. Still, some interesting switching sets may be:

In [32]:
[B for B in Brepresentatives if Rcube.T*B*Rcube == B]

[
[0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 1 1]  [0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 1 1]  [0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0]  [0 0 0 0 1 1 0 0]  [0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0]  [0 0 0 0 1 1 0 0]  [0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0]  [0 0 1 1 0 0 0 0]  [1 1 1 1 0 0 0 0]
[0 0 0 0 0 0 0 0]  [0 0 1 1 0 0 0 0]  [1 1 1 1 0 0 0 0]
[0 0 0 0 0 0 0 0]  [1 1 0 0 0 0 0 0]  [1 1 1 1 0 0 0 0]
[0 0 0 0 0 0 0 0], [1 1 0 0 0 0 0 0], [1 1 1 1 0 0 0 0],

[0 0 0 1 0 1 0 1]  [0 0 1 1 1 1 1 1]
[0 0 1 0 1 0 1 0]  [0 0 1 1 1 1 1 1]
[0 1 0 0 0 1 0 1]  [1 1 0 0 1 1 1 1]
[1 0 0 0 1 0 1 0]  [1 1 0 0 1 1 1 1]
[0 1 0 1 0 0 0 1]  [1 1 1 1 0 0 1 1]
[1 0 1 0 0 0 1 0]  [1 1 1 1 0 0 1 1]
[0 1 0 1 0 1 0 0]  [1 1 1 1 1 1 0 0]
[1 0 1 0 1 0 0 0], [1 1 1 1 1 1 0 0]
]

All (reducible) switching sets for Cube switching:

In [33]:
Brepresentatives

[
[0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]  [0 0 0 0 1 1 1 0]  [0 0 0 1 1 0 1 0]
[0 0 0 0 0 0 0 0]  [0 0 0 0 0 1 0 1]  [0 0 0 1 0 1 0 1]
[0 0 0 0 0 0 0 0]  [0 0 0 0 1 0 1 1]  [0 1 1 0 1 0 0 1]
[0 0 0 0 0 0 0 0]  [0 1 0 1 0 0 0 0]  [0 1 0 1 0 1 0 0]
[0 0 0 0 0 0 0 0]  [0 1 1 0 0 0 1 0]  [0 0 1 0 1 0 0 0]
[0 0 0 0 0 0 0 0]  [0 1 0 1 0 1 0 1]  [0 1 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0], [0 0 1 1 0 0 1 0], [0 0 1 1 0 0 1 0],

[0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0]
[0 0 0 1 1 1 1 0]  [0 0 1 0 1 0 1 0]  [0 0 1 0 1 0 1 1]
[0 0 0 1 0 1 0 1]  [0 1 0 1 0 0 0 0]  [0 1 0 1 0 0 0 1]
[0 1 1 0 1 1 0 1]  [0 0 1 0 0 0 0 0]  [0 0 1 0 0 0 0 0]
[0 1 0 1 0 0 0 0]  [0 1 0 0 0 1 0 0]  [0 1 0 0 0 1 0 0]
[0 1 1 1 0 0 1 0]  [0 0 0 0 1 0 0 0]  [0 0 0 0 1 0 0 1]
[0 1 0 0 0 1 0 1]  [0 1 0 0 0 0 0 1]  [0 1 0 0 0 0 0 0]
[0 0 1 1 0 0 1 0], [0 0 0 0 0 0 1 0], [0 1 1 0 0 1 0 0],

[0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 1 0]
[0 0 1 0 1 1 1 1]  [0 0 1 1 1 1 1 1]  [0 0